# TPI Teoría de Control: Simulación de Controlador PD

**Alumno:** Matías Ezequiel Nuñez

**Materia:** Teoría de Control (K4572)

Este notebook implementa la simulación dinámica del sistema de **Rate Limiting** modelado como un lazo de control cerrado.
A diferencia del enfoque clásico de Token Bucket (PI), aquí implementamos:

1.  **Controlador PD:** $G_c(s) = K_p + K_d \cdot s$.
2.  **Actuador con Memoria:** El mecanismo de asignación de recursos (Bucket/Autoscaler) actúa como un integrador puro en el lazo directo.
3.  **Realimentación Unitaria:** $H(s) = 1$.

El objetivo es validar que el sistema es estable y presenta error estacionario nulo ($e_{ss}=0$) gracias a la naturaleza "Tipo 1" del lazo completo, a pesar de que el controlador es PD.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, RadioButtons
from IPython.display import display

# Configuración de Matplotlib para Google Colab
%matplotlib inline
plt.style.use('seaborn-v0_8-darkgrid')

In [ ]:
class SimuladorRateLimiter:
    def __init__(self):
        self.dt = 0.05  # Paso de tiempo
        self.sim_time = 30.0 # Duración
        self.t = np.arange(0, self.sim_time, self.dt)
        self.n = len(self.t)
        
        # Valores iniciales del controlador (Default)
        self.Kp = 0.5
        self.Kd = 0.2
        
        # Escenario actual
        self.escenario = 'Rafagas' # 'Rafagas' o 'DoS'

    def generar_escenario(self):
        """Genera las señales de Referencia (Setpoint) y Perturbación (D)"""
        # Referencia (R): Nivel de servicio deseado (ej. 100 req/s)
        R = np.ones(self.n) * 100.0 
        
        # Perturbación (D): Tráfico entrante NO deseado o carga extra
        D = np.zeros(self.n)
        
        if self.escenario == 'Rafagas':
            # Ráfagas cortas en t=5s y t=15s
            D[int(5/self.dt):int(7/self.dt)] = 150.0  # Pico fuerte
            D[int(15/self.dt):int(18/self.dt)] = 80.0 # Pico medio
            
        elif self.escenario == 'DoS':
            # Ataque sostenido desde t=5s hasta el final
            D[int(5/self.dt):] = 400.0 # Carga masiva constante
            
        return R, D

    def ejecutar_simulacion(self):
        R, D = self.generar_escenario()
        
        # Inicialización de vectores
        Y = np.zeros(self.n) # Salida (Throughput real)
        e = np.zeros(self.n) # Error
        u = np.zeros(self.n) # Señal de control
        
        # Variables de estado
        capacidad_actual = 0.0 # Variable interna del actuador (Memoria/Bucket)
        y_prev = 0.0
        e_prev = 0.0
        
        # Bucle de simulación
        for i in range(1, self.n):
            # 1. Medición y Cálculo del Error (H(s)=1)
            e[i] = R[i] - y_prev
            
            # 2. Controlador PD: u(t) = Kp*e(t) + Kd*de(t)/dt
            derivativa = (e[i] - e_prev) / self.dt
            u[i] = (self.Kp * e[i]) + (self.Kd * derivativa)
            
            # 3. Actuador con "Memoria" (Efecto Integrador)
            # Acumula la señal de control u(t). Esto convierte al sistema en Tipo 1.
            capacidad_actual += u[i] * self.dt * 5.0 # Factor de ganancia
            
            # Limitaciones físicas (No hay capacidad negativa)
            if capacidad_actual < 0: capacidad_actual = 0
            
            # 4. Planta (Lag de primer orden) + Perturbación
            tau = 0.5 
            entrada_planta = capacidad_actual - D[i] * 0.2 
            Y[i] = (self.dt * entrada_planta + tau * y_prev) / (tau + self.dt)
            
            # Actualizar previos
            y_prev = Y[i]
            e_prev = e[i]
            
        return self.t, R, Y, u, e, D

## Laboratorio Interactivo

Utilice los controles a continuación para modificar los parámetros del controlador en tiempo real:

* **Sliders Kp / Kd:** Ajuste las ganancias Proporcional y Derivativa. Observe cómo un $K_d$ mayor amortigua las oscilaciones.
* **Radio Buttons:** Cambie entre escenarios:
    * *Ráfagas:* Evalúa la respuesta transitoria.
    * *Ataque DoS:* Evalúa la estabilidad y el error en estado estacionario.

In [ ]:
sim = SimuladorRateLimiter()

def plot_simulacion(Kp, Kd, escenario):
    """Función para generar y mostrar la simulación con los parámetros dados"""
    sim.Kp = Kp
    sim.Kd = Kd
    sim.escenario = escenario
    
    t, R, Y, u, e, D = sim.ejecutar_simulacion()
    
    # Crear figura más grande para aprovechar la pantalla
    fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(14, 10), sharex=True)
    
    # Gráfica 1: Respuesta del sistema (con labels exactos del original)
    ax1.plot(t, R, 'k--', label=r'$\theta_i$ (Setpoint / R)', linewidth=1.5)
    ax1.plot(t, Y, 'b-', label='Respuesta del Sistema (Y)', linewidth=2)
    ax1.plot(t, D, 'r:', label='Perturbación (D)', alpha=0.6)
    ax1.set_title('Respuesta Temporal del Sistema (Rate Limiter Controlado)')
    ax1.set_ylabel('Throughput (req/s)')
    ax1.legend(loc='upper right')
    ax1.grid(True, linestyle=':', alpha=0.6)
    
    # Gráfica 2: Error
    ax2.plot(t, e, 'g-', label='Error (e)', linewidth=1.5)
    ax2.set_ylabel('Error $e(t)$')
    ax2.grid(True, linestyle=':', alpha=0.6)
    
    # Gráfica 3: Señal de control
    ax3.plot(t, u, 'm-', label='Salida Controlador (u)', linewidth=1.5)
    ax3.set_ylabel('Señal de Control $u(t)$')
    ax3.set_xlabel('Tiempo (s)')
    ax3.grid(True, linestyle=':', alpha=0.6)
    
    plt.tight_layout()
    plt.show()

# Crear widgets interactivos compatibles con Google Colab
# Usando los mismos rangos que el archivo original: 0-5.0 con step 0.1
interact(plot_simulacion,
         Kp=FloatSlider(min=0.0, max=5.0, step=0.1, value=0.5, description='Kp (Prop.)'),
         Kd=FloatSlider(min=0.0, max=5.0, step=0.1, value=0.2, description='Kd (Deriv.)'),
         escenario=RadioButtons(options=['Rafagas', 'DoS'], value='Rafagas', description='Escenario:'));

### Conclusiones de la Simulación

**1. Escenario Ráfagas (Transitorio):**
Se observa cómo el sistema reacciona ante picos de tráfico.
- Al aumentar **Kp**, el sistema reacciona más rápido pero puede presentar sobrepicos (overshoot).
- Al aumentar **Kd**, se introduce amortiguamiento, suavizando la respuesta y reduciendo oscilaciones, validando la acción derivativa del controlador PD.

**2. Escenario Ataque DoS (Estado Estacionario):**
Ante una perturbación sostenida (escalón):
- La salida $Y(t)$ regresa al valor de referencia (Setpoint).
- El error $e(t)$ converge a **CERO**.
- **Justificación Teórica:** Aunque el controlador es PD (sin término integral explícito), el actuador posee memoria (acumulación de recursos/tokens). Esto añade un polo en el origen al lazo abierto, convirtiendo al sistema en **Tipo 1**, lo que garantiza $e_{ss}=0$ ante entradas escalón.